In [55]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

import joblib

1. 불필요한 컬럼 제거 (특별공급 이런거 어떻게 할지 고민)
    - 각 데이터 셋 별로 합칠거 고려해서 따로 함수 구현해야됨

2. 실수형 데이터 파이프 라인
    - imputer
    - 추가 특성 생성
    - 스케일러

3. 범주형 데이터 변환 후 합치기 
    - 범주형 데이터 변환 뭐로 할거야?

In [161]:
df = pd.read_csv("./src/datasets/병합_청약매물_목록_정보.csv", encoding='cp949')

In [142]:
# 당첨자 d
df_prize = pd.read_csv("./src/datasets/한국부동산원_지역별 청약 당첨자 정보_20250131.csv", encoding='cp949')

# 신청자 d
df_app = pd.read_csv("./src/datasets/한국부동산원_지역별 청약 신청자 정보_20250131.csv", encoding='cp949')

# 가점정보 d nan값 있음
df_plus = pd.read_csv("./src/datasets/한국부동산원_청약홈_APT 가점정보_20250225.csv", encoding='cp949')

# 경쟁률 d nan값 있음 -> 0으로 채울거
df_fight = pd.read_csv("./src/datasets/한국부동산원_청약홈_APT 경쟁률_20250225.csv", encoding='cp949')

# 분양정보 d
df_info = pd.read_csv("./src/datasets/한국부동산원_청약홈_APT 분양정보_20250225.csv", encoding='cp949')

# 청약통장 d
df_account = pd.read_csv("./src/datasets/한국부동산원_청약홈_청약통장_전체 가입현황_20231231.csv", encoding='cp949')


In [144]:
# 주택구분코드와 주택상세구분코드가 다른 행을 찾기
different_rows = df_info[df_info['주택구분코드'] != df_info['주택상세구분코드']]

# 해당 행의 개수 계산
count_different_rows = len(different_rows)

print("주택구분코드와 주택상세구분코드가 다른 행의 개수:", count_different_rows)


주택구분코드와 주택상세구분코드가 다른 행의 개수: 293


### 불필요 컬럼 제거

분양정보

In [145]:
# 민간주택만 필터링 (공공주택은 가점제 없음)
df_info = df_info[df_info['주택상세구분코드명'] != '국민']

# 불필요 칼럼 확인

unnessary_columns = []

# # Case 1) 칼럼 값이 하나라면 불필요 칼럼
# for column in df_info.columns:
#     print(f"{column}: {len(df_info[column].unique())}")

#     if len(df_info[column].unique()) == 1:
#         unnessary_columns.append(column)

# Case 2) 홈페이지 주소, 문의처 칼럼
for column in df_info.columns:
    if column in ['홈페이지주소', '모집공고홈페이지주소', '문의처']:
        unnessary_columns.append(column)

# Case 3) 특별공급 관련 칼럼
for column in df_info.columns:
    if '특별공급' in column:
        unnessary_columns.append(column)

# Case 4) 청약접수종료일을 제외한 나머지 일자 칼럼
for column in df_info.columns:
    if column != '청약접수종료일':
        if ('시작일' in column or '종료일' in column):
            unnessary_columns.append(column)

# Case 5) 그 외 기타 칼럼
unnessary_columns.extend(['주택관리번호', '공급위치우편번호', '모집공고일', '당첨자발표일', '입주예정월'])

# 시공사랑 시행사는 불필요할것같아서 추가로 제거할게요
unnessary_columns.extend(['건설업체명_시공사', '사업주체명_시행사'])

print(unnessary_columns)

df_info = df_info.drop(columns=unnessary_columns)



['홈페이지주소', '문의처', '모집공고홈페이지주소', '특별공급접수시작일', '특별공급접수종료일', '청약접수시작일', '특별공급접수시작일', '특별공급접수종료일', '해당지역1순위접수시작일', '해당지역1순위접수종료일', '경기지역1순위접수시작일', '경기지역1순위접수종료일', '기타지역1순위접수시작일', '기타지역1순위접수종료일', '해당지역2순위접수시작일', '해당지역2순위접수종료일', '경기지역2순위접수시작일', '경기지역2순위접수종료일', '기타지역2순위접수시작일', '기타지역2순위접수종료일', '계약시작일', '계약종료일', '주택관리번호', '공급위치우편번호', '모집공고일', '당첨자발표일', '입주예정월', '건설업체명_시공사', '사업주체명_시행사']


가점정보, 경쟁률

In [146]:
# 주택관리번호, 모델번호

df_plus = df_plus.drop(["주택관리번호", "모델번호"], axis=1)
df_fight = df_fight.drop(["주택관리번호", "모델번호"], axis=1)

### 전처리

주택형 컬럼 분리 :경쟁률, 가점정보

In [147]:
def split_housing_type(df):
    if '주택형' in df.columns:
        # Extract the '전용면적' (before the '.') and remove leading zeros
        df['전용면적'] = df['주택형'].apply(lambda x: int(x.split('.')[0].lstrip('0')))
        
        # Extract the '평면유형' (last character)
        df['평면유형'] = df['주택형'].apply(lambda x: x[-1])
        
        # Drop the original '주택형' column
        df = df.drop(columns=['주택형'])
    
    return df

In [148]:
split_housing_type(df_plus)

,공고번호,거주코드,거주지역,최저당첨가점,최고당첨가점,평균당첨가점,전용면적,평면유형
0,2025000004,1,해당지역,NaN,NaN,NaN,59,A
1,2025000004,2,기타지역,NaN,NaN,NaN,59,A
2,2025000004,1,해당지역,NaN,NaN,NaN,59,B
3,2025000004,2,기타지역,NaN,NaN,NaN,59,B
4,2025000004,1,해당지역,NaN,NaN,NaN,75,A
...,...,...,...,...,...,...,...,...
23090,2020000001,2,기타지역,NaN,NaN,NaN,59,H
23091,2020000001,1,해당지역,NaN,NaN,NaN,84,H
23092,2020000001,2,기타지역,NaN,NaN,NaN,84,H
23093,2020000001,1,해당지역,NaN,NaN,NaN,84,N


경쟁률 미달 처리, NaN값 처리  
    - 근데 NaN값 처리는.. 걍 함수로 만드는게 낫지않나..

In [149]:
import re

def preprocessing_applicant_rate(row):
    # 미달인 경우 경쟁률 처리 (접수건수 - 미달수) / 접수건수
    if '△' in str(row['경쟁률']):
        pattern = '[^0-9]'
        shortage = int(re.sub(pattern, '', row['경쟁률']))

        if row['접수건수'] == 0:
            return 0

        rate = round((row['접수건수'] - shortage) / row['접수건수'], 2)
        return rate

    # 쉼표 제거 후 float으로 변환
    else:
        return float(str(row['경쟁률']).replace(',', ''))


In [150]:
df_fight['경쟁률'] = df_fight.apply(preprocessing_applicant_rate, axis=1)

# '미달여부' 열을 생성
df_fight['미달여부'] = df_fight.apply(lambda x: 'Y' if float(x['경쟁률']) < 1 else 'N', axis=1)

# df_fight['경쟁률'] = df_fight['경쟁률'].fillna(0)


In [151]:
df_plus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23095 entries, 0 to 23094
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   공고번호    23095 non-null  int64  
 1   주택형     23095 non-null  object 
 2   거주코드    23095 non-null  int64  
 3   거주지역    23095 non-null  object 
 4   최저당첨가점  12678 non-null  float64
 5   최고당첨가점  12678 non-null  float64
 6   평균당첨가점  12678 non-null  float64
 7   전용면적    23095 non-null  int64  
 8   평면유형    23095 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 1.6+ MB


NaN값 처리

In [156]:
# 경쟁률 nan값에만 쓸것...

def fill_nan_with_zero(df):
    return df.fillna(0)

In [157]:
df_fight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44228 entries, 0 to 44227
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   공고번호    44228 non-null  int64  
 1   주택형     44228 non-null  object 
 2   공급세대수   44228 non-null  int64  
 3   순위      44228 non-null  int64  
 4   거주코드    44228 non-null  int64  
 5   거주지역    44228 non-null  object 
 6   접수건수    44228 non-null  int64  
 7   경쟁률     21928 non-null  float64
 8   미달여부    44228 non-null  object 
dtypes: float64(1), int64(5), object(3)
memory usage: 3.0+ MB


### 데이터 파이프 라인 준비

실수형

In [158]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

# FunctionTransformer로 함수를 변환
nan_transformer = FunctionTransformer(fill_nan_with_zero)

# 파이프라인 구성
num_pipeline = Pipeline([
    ("nan", nan_transformer),
    ("std_scaler", StandardScaler())
])

범주형